In [1]:
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import viridis

/Users/rutujajangle/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('Combined.csv')
duration_mins_secs = [str(f"{i}:{j}") for i, j in zip([int(i//1) for i in list(df['Duration'])], [int(round((i%1) * 60, 2)) for i in list(df['Duration'])])]
df['Track Duration'] = duration_mins_secs
df.sample(5)

,Unnamed: 0,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,...,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Playlist,Track Duration
74,74,Heat Waves,spotify:track:3USxtqRwSYz57Ewm6wWRMp,https://open.spotify.com/track/3USxtqRwSYz57Ew...,['Glass Animals'],Dreamland,90,3.98,80.870,11,...,0.525,-6.900,0.0944,0.4400,0.000007,0.0921,0.531,4,Playlist 2,3:58
46,46,Bohemian Rhapsody - Remastered 2011,spotify:track:4u7EnebtmKWzUH433cf5Qv,https://open.spotify.com/track/4u7EnebtmKWzUH4...,['Queen'],A Night At The Opera (2011 Remaster),83,5.91,71.105,0,...,0.404,-9.928,0.0499,0.2710,0.000000,0.3000,0.224,4,Playlist 1,5:54
87,87,Closer,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"['The Chainsmokers', 'Halsey']",Closer,86,4.08,95.010,8,...,0.524,-5.599,0.0338,0.4140,0.000000,0.1110,0.661,4,Playlist 2,4:4
71,71,Paris,spotify:track:72jbDTw1piOOj770jWNeaG,https://open.spotify.com/track/72jbDTw1piOOj77...,['The Chainsmokers'],Memories...Do Not Open,77,3.69,99.990,2,...,0.658,-6.428,0.0304,0.0215,0.000002,0.0939,0.219,4,Playlist 2,3:41
40,40,Secrets,spotify:track:1NhPKVLsHhFUHIOZ32QnS2,https://open.spotify.com/track/1NhPKVLsHhFUHIO...,['OneRepublic'],Waking Up,79,3.74,148.021,2,...,0.764,-6.223,0.0366,0.0717,0.000000,0.1150,0.376,4,Playlist 1,3:44


In [3]:
p1 = pd.read_csv('Playlist1.csv')
p1 = pd.DataFrame(p1[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Speechiness', 'Valence']].mean()).T
p2 = pd.read_csv('Playlist2.csv')
p2 = pd.DataFrame(p2[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Speechiness', 'Valence']].mean()).T
p = pd.concat([p1, p2], ignore_index=True)
p

,Acousticness,Danceability,Energy,Instrumentalness,Speechiness,Valence
0,0.323171,0.49882,0.54488,0.051619,0.035736,0.346190
1,0.134577,0.64820,0.71412,0.012604,0.061402,0.557222


In [4]:
artists = pd.read_csv('All Artists.csv')
artists['Index'] = artists.index
artists.sample(5)

,Name,ID,Genres,Followers,Popularity,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2,Index
70,Maren Morris,6WY7D3jk8zTrHtmkqqo5GI,"['classic texas country', 'contemporary country']",777453,68,Playlist 2,1,0,1,70
55,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,['australian hip hop'],4927850,81,Playlist 2,1,0,1,55
30,Coldplay,4gzpq5DPGxSnKTe4SA8HAU,"['permanent wave', 'pop']",48485132,86,Both,5,3,2,30
80,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,"['art pop', 'dance pop', 'pop']",28632759,82,Playlist 2,1,0,1,80
53,DJ Snake,540vIaP2JwjQb9dm3aArA4,"['dance pop', 'edm', 'electronic trap', 'pop',...",8779692,76,Playlist 2,1,0,1,53


In [5]:
genres = pd.read_csv('Genres.csv')
genres['Index'] = genres.index
genres.sample(5)

,Genre,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2,Index
69,madchester,Playlist 1,1,1,0,69
79,british invasion,Playlist 1,1,1,0,79
38,baroque pop,Playlist 1,1,1,0,38
89,escape room,Playlist 2,1,0,1,89
25,australian dance,Playlist 2,1,0,1,25


In [6]:
collabs = pd.read_csv('All Collaborations.csv')
collabs.sample(5)

,Artist 1,Artist 2,Popular Collab,URL,Popularity,Artist From
64,Mark Ronson,Camila Cabello,Find U Again (feat. Camila Cabello),https://open.spotify.com/track/1HpzOCZbNWzxvvX...,56.0,Artists from Same Playlist
37,BTS,Zara Larsson,A Brand New Day (BTS World Original Soundtrack...,https://open.spotify.com/track/3amNzwQ9muL0ZwD...,59.0,Artists from Same Playlist
65,Mark Ronson,Miley Cyrus,Nothing Breaks Like a Heart (feat. Miley Cyrus),https://open.spotify.com/track/02dRkCEc8Q5ch4T...,77.0,Artists from Same Playlist
79,Dua Lipa,Mark Ronson,Physical (feat. Gwen Stefani) [Mark Ronson Rem...,https://open.spotify.com/track/1GGuz6FiDdy6uH5...,45.0,Artists from Same Playlist
130,Lady Gaga,The Weeknd,Marry The Night - The Weeknd & Illangelo Remix,https://open.spotify.com/track/5bfivmKwgSni611...,28.0,Artists from Same Playlist


In [7]:
artists_songs = pd.read_csv("Common Artist Songs.csv")
artists_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Artist
8,Emperor's New Clothes,spotify:track:3px2rAPu74ltbkf9eZsZ8h,https://open.spotify.com/track/3px2rAPu74ltbkf...,['Panic! At The Disco'],Death of a Bachelor,71,2.64,94.481,1,1,0.562,0.904,-4.947,0.0807,0.00302,0.000000,0.0433,0.651,4,Panic! At The Disco
3,Lover,spotify:track:1dGr1c8CrMLDpV6mPbImSI,https://open.spotify.com/track/1dGr1c8CrMLDpV6...,['Taylor Swift'],Lover,92,3.69,68.534,7,1,0.359,0.543,-7.582,0.0919,0.49200,0.000016,0.1180,0.453,4,Taylor Swift
12,Shivers,spotify:track:50nfwKoDiSYg8zOCREWAm5,https://open.spotify.com/track/50nfwKoDiSYg8zO...,['Ed Sheeran'],=,86,3.46,141.020,2,1,0.788,0.859,-2.724,0.0856,0.28100,0.000000,0.0424,0.822,4,Ed Sheeran
11,Shape of You,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://open.spotify.com/track/7qiZfU4dY1lWllz...,['Ed Sheeran'],÷ (Deluxe),88,3.90,95.977,1,0,0.825,0.652,-3.183,0.0802,0.58100,0.000000,0.0931,0.931,4,Ed Sheeran
4,"""Slut!"" (Taylor's Version) (From The Vault)",spotify:track:71BqAINEnezjQfxE4VuJfq,https://open.spotify.com/track/71BqAINEnezjQfx...,['Taylor Swift'],1989 (Taylor's Version),91,3.01,77.983,7,1,0.629,0.413,-13.178,0.0550,0.34500,0.000076,0.0603,0.306,4,Taylor Swift


In [8]:
genre_songs = pd.read_csv("Common Genre Songs.csv")
genre_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Genre
187,Walking On A Dream,spotify:track:3HfhB8sYqLlVmpBPb7cc2x,https://open.spotify.com/track/3HfhB8sYqLlVmpB...,['Empire of the Sun'],Walking On A Dream,65,3.31,126.967,5,0,0.870,0.699,-5.541,0.0459,0.245000,0.000007,0.0588,0.726,4,indietronica
36,Smells Like Teen Spirit,spotify:track:5ghIJDpPoe3CfHMGu71E6T,https://open.spotify.com/track/5ghIJDpPoe3CfHM...,['Nirvana'],Nevermind (Remastered),80,5.03,116.761,1,1,0.502,0.912,-4.556,0.0564,0.000025,0.000173,0.1060,0.720,4,permanent wave
241,Why'd You Only Call Me When You're High?,spotify:track:086myS9r57YsLbJpU0TgK9,https://open.spotify.com/track/086myS9r57YsLbJ...,['Arctic Monkeys'],AM,90,2.69,92.004,2,1,0.691,0.631,-6.478,0.0368,0.048300,0.000011,0.1040,0.800,4,rock
250,In the End,spotify:track:60a0Rd6pjrkxjPbaKzXjfq,https://open.spotify.com/track/60a0Rd6pjrkxjPb...,['Linkin Park'],Hybrid Theory (Bonus Edition),89,3.61,105.143,3,0,0.556,0.864,-5.870,0.0584,0.009580,0.000000,0.2090,0.400,4,rock
94,What Am I To You?,spotify:track:0AMKZWRgdFfGYYTxiWrwwH,https://open.spotify.com/track/0AMKZWRgdFfGYYT...,['Norah Jones'],Feels Like Home,67,3.50,75.978,0,1,0.651,0.347,-9.175,0.0341,0.694000,0.024900,0.1100,0.533,4,neo mellow


In [9]:
new_songs = pd.read_csv("New Playlist.csv")
duration_mins_secs = [str(f"{i}:{j}") for i, j in zip([int(i//1) for i in list(new_songs['Duration'])], [int(round((i%1) * 60, 2)) for i in list(new_songs['Duration'])])]
new_songs['Track Duration'] = duration_mins_secs
new_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Track Duration
104,First Love/Late Spring,spotify:track:3sslYZcFKtUvIEWN9lADgr,https://open.spotify.com/track/3sslYZcFKtUvIEW...,['Mitski'],Bury Me At Makeout Creek,81,4.64,99.963,0,1,0.445,0.543,-8.196,0.0272,0.000825,0.013700,0.2020,0.102,4,4:38
186,Everything She Wants,spotify:track:5hXEcqQhEjfZdbIZLO8mf2,https://open.spotify.com/track/5hXEcqQhEjfZdbI...,['Wham!'],Make It Big,71,5.04,115.090,6,0,0.899,0.459,-15.557,0.0366,0.322000,0.004060,0.1650,0.962,4,5:2
4,Lover,spotify:track:1dGr1c8CrMLDpV6mPbImSI,https://open.spotify.com/track/1dGr1c8CrMLDpV6...,['Taylor Swift'],Lover,92,3.69,68.534,7,1,0.359,0.543,-7.582,0.0919,0.492000,0.000016,0.1180,0.453,4,3:41
71,Softcore,spotify:track:2K7xn816oNHJZ0aVqdQsha,https://open.spotify.com/track/2K7xn816oNHJZ0a...,['The Neighbourhood'],Hard To Imagine The Neighbourhood Ever Changing,87,3.44,93.986,9,0,0.575,0.568,-5.509,0.0300,0.048400,0.000417,0.2860,0.370,4,3:26
358,Come Together - Remastered 2009,spotify:track:2EqlS6tkEnglzr7tkKAAYD,https://open.spotify.com/track/2EqlS6tkEnglzr7...,['The Beatles'],Abbey Road (Remastered),80,4.33,165.007,9,0,0.533,0.376,-11.913,0.0393,0.030200,0.248000,0.0926,0.187,4,4:19


In [10]:
cols = ['Popularity', 'Tempo', 'Danceability', 'Energy', 'Acousticness', 'Valence']
scatter_cols = ['Acousticness', 'Danceability', 'Energy', 'Tempo', 'Valence']
hist_cols = ['Popularity', 'Tempo', 'Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveliness', 'Valence']

---

In [11]:
app1 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app1.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='left', children=[
            dbc.Label("Comparing Playlists - CDF Plots"),
            dcc.Graph(id="histogram")
        ]),
        dbc.Col(width=2, align='right', children=[
            html.Div([
                html.Label('Field'),
                dcc.Dropdown(id='field', value=hist_cols[0], options=[{'label': col, 'value': col} for col in hist_cols])
            ]),
            html.Div([
                html.Label('Bins'),
                dcc.Slider(id='nbins', min=5, max=200, marks=None, step=1, value=10)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, marks=None, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app1.callback(
    Output("histogram", "figure"), 
    [Input("field", "value"),
     Input('nbins', 'value'), 
     Input('opacity', 'value')]
)

def update_chart(field, nbins, opacity):
    fig = px.histogram(df, x=field, histfunc='count', nbins=nbins, color='Playlist', color_discrete_sequence=['cyan', 'lightpink'], opacity=opacity, cumulative=True, width=1400, height=600)
    fig.update_layout(
        barmode='overlay',
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app1.run(port=8855, debug=True)

---

In [12]:
app2 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app2.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Scatterplot"),
            dcc.Graph(id="scatter")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('X-Axis'),
                dcc.Dropdown(id='xax', value=scatter_cols[0], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Y-Axis'),
                dcc.Dropdown(id='yax', value=scatter_cols[1], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Size'),
                dcc.Dropdown(id='size', value=scatter_cols[2], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app2.callback(
    Output("scatter", "figure"), 
    [Input("xax", "value"),
     Input('yax', 'value'),
     Input('size', 'value'),
     Input('opacity', 'value')]
)

def update_figure(xax, yax, size, opacity):
    hover_dict = {xax: False, yax:False, size:False, 'Playlist':False, 'Album':True, 'Artist(s)':True, 'Track Duration':True, 'Popularity':True}
    fig = px.scatter(df, x=xax, y=yax, size=size, color='Playlist', trendline='ols', color_discrete_sequence=['cyan', 'lightpink'], hover_name="Track_Name", hover_data=hover_dict, size_max=25, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity=opacity)
    return fig

app2.run(port=8856, debug=True)

---

In [15]:
app3 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app3.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Radar Plot"),
            dcc.Graph(id="radar"),
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5, marks=None, step=None)
            ]),
        ])
    ])
], fluid=True)

@app3.callback(
    Output("radar", "figure"),
    Input('opacity', 'value')
)

def update_figure(opacity):
    trace1 = go.Scatterpolar(
        r = list(p.iloc[0, :]),
        theta = list(p.columns),
        fill = 'toself',
        fillcolor = 'cyan',
        line = dict(color='black', width=1),
        name='Playlist 1'
    )
    trace2 = go.Scatterpolar(
        r = list(p.iloc[1, :]),
        theta = list(p.columns),
        fill = 'toself',
        fillcolor = 'lightpink',
        line = dict(color='black', width=1),
        name = 'Playlist 2'
    )
    layout = go.Layout(
        polar = dict(
            radialaxis = dict(
                visible = True,
                range = [0, 1]
            )
        ),
        showlegend = True,
        height = 600,
        width = 1400,
        plot_bgcolor = 'rgba(0, 0, 0, 0)'
    )
    fig = go.Figure(data=[trace1, trace2], layout=layout)
    fig.update_layout(
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity = opacity)
    return fig

app3.run(port=8857, debug=True)

---

In [19]:
app4 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app4.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Top Artists"),
            dcc.Graph(id="bubble")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value='Followers', options=[{'label': col, 'value': col} for col in ['Followers', 'Popularity']])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app4.callback(
    Output("bubble", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value'),
     Input('opacity', 'value')]
)

def update_figure(measure, topn, opacity):
    hover_dict = {measure: False, 'Index':False, 'Name':False, 'Playlist':False, 'Genres':True, 'Popularity':True, 'Followers':True, 'Total Count':False}
    fig = px.scatter(artists.sort_values(by=measure, ascending=False).head(topn), x='Index', y=measure, text='Name', size='Total Count', size_max=100, color='Playlist', color_discrete_map={'Playlist 1':'cyan', 'Playlist 2':'lightpink', 'Both':'purple'}, hover_name="Name", hover_data=hover_dict, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app4.run(port=8858, debug=True)

---

In [20]:
app5 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app5.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Top Genres"),
            dcc.Graph(id="bubble_too")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app5.callback(
    Output("bubble_too", "figure"),
     [Input('topn', 'value'),
     Input('opacity', 'value')]
)

def update_figure(topn, opacity):
    hover_dict = {'Index':False, 'Genre':False, 'Playlist':False, 'Total Count':True}
    fig = px.scatter(genres.sort_values(by='Total Count', ascending=False).head(topn), x='Total Count', y='Index', text='Genre', size='Total Count', size_max=100, color='Playlist', color_discrete_map={'Playlist 1':'cyan', 'Playlist 2':'lightpink', 'Both':'purple'}, hover_name="Genre", hover_data=hover_dict, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel=dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app5.run(port=8859, debug=True)

---

In [21]:
app6 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app6.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Collaboration Network"),
            dcc.Graph(id="network")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ])
        ])
    ])
], fluid=True)

@app6.callback(
    Output("network", "figure"),
     Input('opacity', 'value')
)

def update_figure(opacity):

    G = nx.Graph()
    for i, j, k, l, m in zip(list(artists['Name']), list(artists['Genres']), list(artists['Popularity']), list(artists['Followers']), list(artists['Playlist'])):
        G.add_node(i)
        G.nodes[i]['Genres'] = j
        G.nodes[i]['Popularity'] = k
        G.nodes[i]['Followers'] = l
        G.nodes[i]['Playlist'] = m
    for i, j, k, l, m in zip(list(collabs['Artist 1']), list(collabs['Artist 2']), list(collabs['Popular Collab']), list(collabs['Popularity']), list(collabs['Artist From'])):
        G.add_edge(i, j)
        G.edges[(i, j)]['Popular Collab'] = k
        G.edges[(i, j)]['Popularity'] = l
        G.edges[(i, j)]['Artist From'] = m
    #nx.write_gexf(G, 'Collaboration Graph.gexf')

    pos = nx.spring_layout(G, k=0.99)
    pos = nx.rescale_layout_dict(pos, 1000)
    for n in G.nodes:
        G.nodes[n]['pos'] = pos[n]

    edge_x = []
    edge_y = []
    xtext = []
    ytext = []
    edgetext = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        xtext.append((x0 + x1)/2)
        ytext.append((y0 + y1)/2)
        edgetext.append(f"Artist 1: {edge[0]}, \nArtist 2: {edge[1]},\n Collaboration: {G.edges[edge]['Popular Collab']}")
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, 
        y=edge_y,
        line = dict(
            width = 1, 
            color = '#888'
        ),
        hoverinfo='text',
        mode='lines'
    )
    edge_trace.text = edgetext

    eweights_trace = go.Scatter(
        x = xtext,
        y = ytext, 
        mode = 'markers',
        marker = dict(
            size = 3,
            color = '#888',
        ),
        text = edgetext,
        hoverinfo = 'text'
    )

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
    
    node_trace = go.Scatter(
        x = node_x, 
        y = node_y,
        mode = 'markers',
        hoverinfo = 'text',
        marker = dict(
            color = [],
            size = [],
            line_width = 0.5
        )
    )

    pd = {'Playlist 1':'cyan', 'Playlist 2':'lightpink', 'Both':'purple'}

    node_size = []
    node_color = []
    node_text = []
    for n in G.nodes:
        node_size.append((nx.degree_centrality(G)[n] * 500) + 10)
        node_color.append(pd[G.nodes[n]['Playlist']])
        node_text.append(f"Name: {n},\nGenres: {G.nodes[n]['Genres']}")

    node_trace.marker.size = node_size
    node_trace.marker.color = node_color
    node_trace.text = node_text

    layout = go.Layout(
        showlegend = False,
        hovermode = 'closest',
        xaxis = dict(
            showgrid = False, 
            zeroline = False, 
            showticklabels = False
        ),
        yaxis = dict(
            showgrid = False, 
            zeroline = False, 
            showticklabels = False
        ),
        height=1100,
        width=1400,
        plot_bgcolor='rgba(0, 0, 0, 0)'
    )

    fig = go.Figure(data=[edge_trace, eweights_trace, node_trace], layout=layout)
    fig.update_layout(
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity = opacity)
    return fig

app6.run(port=8860, debug=True)

---

In [33]:
app7 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app7.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Wall of Recommendations - Common Genres"),
            dcc.Graph(id="treemap")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
        ])
    ])
], fluid=True)

@app7.callback(
    Output("treemap", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value')]
)

def update_figure(measure, topn):
    hover_dict = {'Artist(s)':True, measure:False, 'Track_Name':False}
    fig = px.treemap(
        genre_songs.sort_values(by=measure, ascending=False).head(topn), 
        path = ['Common Genre', 'Track_Name'],
        values = measure,
        color = measure,
        color_continuous_scale=px.colors.sequential.Reds
,
        hover_name = 'Track_Name',
        hover_data = hover_dict,
    )
    fig.update_layout(
        width=1400,
        height=600,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        ),
    )
    return fig

app7.run(port=8861, debug=True)

---

In [34]:
app8 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app8.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Wall of Recommendations - Common Artists"),
            dcc.Graph(id="treemap_too")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
        ])
    ])
], fluid=True)

@app8.callback(
    Output("treemap_too", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value')]
)

def update_figure(measure, topn):
    hover_dict = {'Artist(s)':True, measure:False, 'Track_Name':False}
    fig = px.treemap(
        artists_songs.sort_values(by=measure, ascending=False).head(topn), 
        path = ['Common Artist', 'Track_Name'],
        values = measure,
        color = measure,
        color_continuous_scale=px.colors.sequential.Reds,
        hover_name = 'Track_Name',
        hover_data = hover_dict
    )
    fig.update_layout(
        width=1400,
        height=600,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app8.run(port=8862, debug=True)

---

In [35]:
app9 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app9.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Scatterplot"),
            dcc.Graph(id="scatter")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('X-Axis'),
                dcc.Dropdown(id='xax', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Y-Axis'),
                dcc.Dropdown(id='yax', value=cols[1], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Size'),
                dcc.Dropdown(id='size', value=cols[2], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Color'),
                dcc.Dropdown(id='color', value=cols[3], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5, step=None, marks=None)
            ]),
        ])
    ])
], fluid=True)

@app9.callback(
    Output("scatter", "figure"), 
    [Input("xax", "value"),
     Input('yax', 'value'),
     Input('size', 'value'),
     Input('color', 'value'),
     Input('opacity', 'value')]
)

def update_figure(xax, yax, size, color, opacity):
    hover_dict = {xax: False, yax:False, size:False, color:False, 'Album':True, 'Artist(s)':True, 'Track Duration':True, 'Popularity':True}
    fig = px.scatter(new_songs, x=xax, y=yax, size=size, color=color, trendline='ols', color_continuous_scale='Reds', hover_name="Track_Name", hover_data=hover_dict, size_max=15, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity=opacity)
    return fig

app9.run(port=8863, debug=True)

---

In [38]:
app10 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app10.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='left', children=[
            dbc.Label("New Playlist - CDF Plot"),
            dcc.Graph(id="histogram")
        ]),
        dbc.Col(width=2, align='right', children=[
            html.Div([
                html.Label('Field'),
                dcc.Dropdown(id='field', value=hist_cols[0], options=[{'label': col, 'value': col} for col in hist_cols])
            ]),
            html.Div([
                html.Label('Bins'),
                dcc.Slider(id='nbins', min=5, max=200, marks=None, step=1, value=10)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, marks=None, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app10.callback(
    Output("histogram", "figure"), 
    [Input("field", "value"),
     Input('nbins', 'value'), 
     Input('opacity', 'value')]
)

def update_chart(field, nbins, opacity):
    fig = px.histogram(new_songs, x=field, histfunc='count', nbins=nbins, opacity=opacity, cumulative=True, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app10.run(port=8864, debug=True)

---